In [1]:
from cana.drawing.schema_vis import plot_schemata
from cana.drawing.plot_look_up_table import plot_look_up_table

from automata.schema_search_tools import (
    concatenate,
    check_spread_randomness,
    check_for_duplicates,
    annihilation_generation_rules,
)

from cana.boolean_node import BooleanNode
import pandas as pd
import random
import matplotlib.pyplot as plt

## automata binary

In [2]:
automata = {
    "GKL_BIN": 0b00000000_01011111_00000000_01011111_00000000_01011111_00000000_01011111_00000000_01011111_11111111_01011111_00000000_01011111_11111111_01011111,
    "GKL_STR": "!x_{c}&x_{l1}&x_{l3} | x_{r3}&x_{c} | x_{r1}&x_{c}",
    "GP_BIN": 0b00000101_00000000_01010101_00000101_00000101_00000000_01010101_00000101_01010101_11111111_01010101_11111111_01010101_11111111_01010101_11111111,
    "GP_STR": "x_{r3}&x_{r1}&!x_{c} | x_{r3}&x_{r1}&x_{l1} | x_{r3}&!x_{c}&x_{l1} | x_{r3}&x_{l3} | x_{c}&x_{l3}",
    "GEP_1_BIN": 0b00010001_00000000_01010101_00000000_00010001_00001111_01010101_00001111_00010001_11111111_01010101_11111111_00010001_11111111_01010101_11111111,
    "GEP_1_STR": "x_{r3}&x_{r2}&!x_{c} | x_{r3}&!x_{c}&x_{l1} | x_{r1}&x_{c}&x_{l2} | x_{c}&x_{l3}",
    "GEP_2_BIN": 0b00000000_01010101_00000000_01110111_00000000_01010101_00000000_01110111_00001111_01010101_00001111_01110111_11111111_01010101_11111111_01110111,
    "GEP_2_STR": "x_{r2}&x_{c}&x_{l1} | x_{r1}&!x_{c}&x_{l3} | !x_{c}&x_{l2}&x_{l3} | x_{r3}&x_{c}",
    "Das_BIN": 0b00000111_00000000_00000111_11111111_00001111_00000000_00001111_11111111_00001111_00000000_00000111_11111111_00001111_00110001_00001111_11111111,
    "Das_STR": "x_{r3}&x_{r2}&x_{r1}&x_{l2}&x_{l3} | x_{r2}&!x_{r1}&x_{c}&x_{l2}&x_{l3} | x_{r1}&!x_{c}&!x_{l1}&x_{l3} | x_{r3}&x_{r1}&!x_{c} | x_{r2}&x_{r1}&!x_{c} | x_{r1}&!x_{c}&x_{l2} | x_{c}&x_{l1}",
    "Davis_BIN": 0b00000000_00101111_00000011_01011111_00000000_00011111_11001111_00011111_00000000_00101111_11111100_01011111_00000000_00011111_11111111_00011111,
    "Davis_STR": "!x_{r3}&x_{r2}&x_{c}&!x_{l1}&!x_{l2} | x_{r3}&x_{c}&x_{l1}&!x_{l2} | x_{r3}&x_{r2}&x_{c}&x_{l2} | !x_{r2}&!x_{c}&x_{l1}&x_{l2} | x_{r2}&x_{r1}&x_{l1}&!x_{l3} | !x_{r2}&x_{r1}&x_{l1}&x_{l3} | !x_{r1}&!x_{c}&x_{l1}&x_{l3} | x_{r1}&x_{l1}&x_{l2} | x_{r1}&x_{c}",
    "ABK_BIN": 0b00000101_00000000_01010101_00000101_00000101_00000000_01010101_00000101_01010101_11111111_01010101_11111111_01010101_11111111_01010101_11111111,
    "ABK_STR": "x_{r3}&x_{r1}&!x_{c} | x_{r3}&x_{r1}&x_{l1} | x_{r3}&!x_{c}&x_{l1} | x_{r3}&x_{l3} | x_{c}&x_{l3}",
    "DMC_BIN": 0b00000101_00000100_00000101_10000111_00000101_00000000_00001111_01110111_00000011_01110111_01010101_10000011_01111011_11111111_10110111_01111111,
    "DMC_STR": "!x_{r3}&!x_{r2}&!x_{r1}&x_{c}&x_{l1}&!x_{l2} | !x_{r3}&!x_{r1}&!x_{c}&x_{l1}&x_{l2}&x_{l3} | x_{r3}&!x_{r2}&x_{r1}&!x_{l2}&!x_{l3} | x_{r1}&!x_{c}&x_{l1}&x_{l2}&!x_{l3} | x_{r3}&!x_{c}&x_{l1}&!x_{l2}&x_{l3} | x_{r3}&!x_{r1}&!x_{l1}&x_{l2}&x_{l3} | !x_{r3}&x_{r1}&!x_{l1}&x_{l2}&x_{l3} | x_{r2}&x_{r1}&x_{c}&x_{l1} | x_{r3}&x_{r1}&x_{l1}&x_{l2} | x_{r3}&x_{c}&x_{l1}&x_{l2} | x_{r2}&x_{c}&x_{l1}&x_{l2} | x_{r3}&x_{r1}&!x_{c}&!x_{l3} | x_{r2}&x_{r1}&!x_{l1}&x_{l3} | x_{r3}&x_{c}&!x_{l1}&x_{l3} | x_{r2}&x_{c}&!x_{l1}&x_{l3} | x_{r1}&x_{c}&x_{l2}&x_{l3} | x_{c}&!x_{l1}&x_{l2}&x_{l3} | x_{r2}&x_{l2}&x_{l3}",
    "COE_1_BIN": 0b00000001_00010100_00110000_11010111_00010001_00001111_00111001_01010111_00000101_10110100_11111111_00010111_11110001_01111101_11111001_01010111,
    "COE_1_STR": "!x_{r3}&!x_{r2}&x_{r1}&!x_{c}&x_{l1}&x_{l2} | !x_{r2}&!x_{r1}&x_{c}&x_{l1}&!x_{l2}&!x_{l3} | !x_{r3}&!x_{r1}&x_{c}&!x_{l1}&!x_{l2}&x_{l3} | x_{r3}&x_{r2}&x_{r1}&!x_{c}&!x_{l1} | x_{r3}&x_{r2}&!x_{r1}&x_{c}&!x_{l2} | !x_{r2}&x_{r1}&x_{c}&!x_{l1}&x_{l2} | x_{r2}&x_{r1}&x_{c}&x_{l2}&!x_{l3} | x_{r2}&!x_{r1}&x_{c}&!x_{l1}&x_{l3} | x_{r3}&!x_{r2}&x_{r1}&!x_{l2}&x_{l3} | x_{r3}&!x_{r2}&x_{r1}&x_{c} | x_{r2}&!x_{r1}&!x_{c}&x_{l1} | x_{r2}&x_{r1}&x_{c}&x_{l1} | x_{r3}&x_{r2}&!x_{c}&x_{l2} | x_{r3}&x_{c}&x_{l1}&x_{l2} | !x_{c}&x_{l1}&!x_{l2}&x_{l3} | !x_{r1}&!x_{c}&x_{l2}&x_{l3} | x_{r3}&x_{c}&x_{l2}&x_{l3}",
    "COE_2_BIN": 0b00010100_01010001_00110000_01011100_00000000_01010000_11001110_01011111_00010111_00010001_11111111_01011111_00001111_01010011_11001111_01011111,
    "COE_2_STR": "x_{r3}&!x_{r2}&x_{r1}&!x_{c}&!x_{l1}&!x_{l2} | x_{r3}&x_{r2}&x_{c}&!x_{l1}&!x_{l2} | x_{r2}&!x_{r1}&!x_{c}&x_{l1}&!x_{l2} | !x_{r2}&x_{r1}&x_{c}&x_{l1} | x_{r3}&x_{r2}&!x_{r1}&!x_{l2} | x_{r3}&!x_{r1}&x_{c}&x_{l2} | !x_{r3}&x_{r1}&x_{l1}&x_{l2} | !x_{r2}&!x_{c}&x_{l1}&x_{l2} | x_{r3}&x_{c}&x_{l1}&x_{l2} | x_{r3}&!x_{r1}&x_{c}&!x_{l3} | x_{r2}&x_{r1}&!x_{c}&x_{l3} | !x_{r2}&!x_{c}&x_{l1}&x_{l3} | x_{r3}&x_{c}&x_{l1}&x_{l3} | x_{r2}&x_{r1}&x_{l2}&x_{l3} | x_{r1}&!x_{c}&x_{l2}&x_{l3} | x_{r1}&x_{l1}&x_{l3}",
    "MM401_BIN": 0b11111111_10101010_11111111_10101000_11111111_10101010_11111111_10101000_11110000_10101010_00000000_10101000_00000000_10101010_00000000_10101000,
    "MM401_STR": "!x_{r1}&!x_{c}&!x_{l1}&!x_{l2} | !x_{r3}&!x_{r2}&x_{c} | !x_{r3}&!x_{r1}&x_{c} | !x_{r3}&x_{c}&!x_{l1} | !x_{c}&!x_{l3}",
}

# output_dict = {}
# # convert this into a list of outputs
# for item in automata:
#     if item[-3:] == 'BIN':
#         outputlist = [str(x) for x in bin(automata[item])[2:].zfill(128)]
#         output_dict[item] = outputlist
# print(output_dict)

## automata output list

In [3]:
automata_output_list = {
    "GKL": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "GP": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "GEP_1": [
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "GEP_2": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
    ],
    "Das": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "Davis": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "ABK": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "DMC": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "COE_1": [
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "0",
        "1",
        "1",
        "0",
        "1",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
    ],
    "COE_2": [
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "0",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
    ],
    "MM401": [
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "1",
        "1",
        "1",
        "1",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "0",
        "1",
        "0",
        "1",
        "0",
        "1",
        "0",
        "0",
        "0",
    ],
}

In [34]:
# autoschemata = {}
# for item in automata_output_list:
#     node = BooleanNode.from_output_list(automata_output_list[item])
#     s = node.schemata_look_up_table()
#     autoschemata[item] = s.values.tolist()

# # print(autoschemata)

## automata schemata

In [35]:
autoschemata = {
    "GKL": [
        ["0##0###", 0],
        ["##0#0#0", 0],
        ["###10#0", 0],
        ["##00###", 0],
        ["0###0#0", 0],
        ["1#1###1", 1],
        ["1#10###", 1],
        ["###11##", 1],
        ["1#1#1##", 1],
        ["###1##1", 1],
    ],
    "GP": [
        ["###0##0", 0],
        ["0#0#0##", 0],
        ["0#####0", 0],
        ["0#01###", 0],
        ["0##10##", 0],
        ["##1#1#1", 1],
        ["1#####1", 1],
        ["1##1###", 1],
        ["##10##1", 1],
        ["###01#1", 1],
    ],
    "GEP_1": [
        ["0#0#00#", 0],
        ["###0##0", 0],
        ["00#1###", 0],
        ["##00#0#", 0],
        ["00####0", 0],
        ["0##10##", 0],
        ["000##0#", 0],
        ["0###0#0", 0],
        ["#1##111", 1],
        ["1####11", 1],
        ["#1#11##", 1],
        ["##10##1", 1],
        ["#11#1#1", 1],
        ["1#1###1", 1],
        ["1##1###", 1],
        ["###0#11", 1],
    ],
    "GEP_2": [
        ["0##0###", 0],
        ["###1#00", 0],
        ["#00#0#0", 0],
        ["0#0###0", 0],
        ["0####00", 0],
        ["##01##0", 0],
        ["#0##000", 0],
        ["#0#00##", 0],
        ["11#0###", 1],
        ["111##1#", 1],
        ["11####1", 1],
        ["1##01##", 1],
        ["1###1#1", 1],
        ["##11#1#", 1],
        ["1#1#11#", 1],
        ["###1##1", 1],
    ],
    "Das": [
        ["#010#00", 0],
        ["0#0#0##", 0],
        ["##01#0#", 0],
        ["#001###", 0],
        ["##011#0", 0],
        ["##0#00#", 0],
        ["#00#0##", 0],
        ["000##00", 0],
        ["###00##", 0],
        ["0#01###", 0],
        ["00#0#00", 0],
        ["11##111", 1],
        ["#1#01##", 1],
        ["##1#1#1", 1],
        ["#11#1##", 1],
        ["##11###", 1],
        ["11#101#", 1],
        ["###011#", 1],
        ["1#001##", 1],
        ["11#1#11", 1],
        ["###01#1", 1],
        ["##1#11#", 1],
    ],
    "Davis": [
        ["0##001#", 0],
        ["00##000", 0],
        ["#1#10#0", 0],
        ["#00#0#1", 0],
        ["###1000", 0],
        ["##0#00#", 0],
        ["##110#0", 0],
        ["##00###", 0],
        ["00#0#0#", 0],
        ["10#011#", 0],
        ["00#00##", 0],
        ["001#0#0", 0],
        ["#1#100#", 0],
        ["#10#0#0", 0],
        ["0#1#010", 0],
        ["01##010", 0],
        ["#001#10", 1],
        ["111##11", 1],
        ["101#0#1", 1],
        ["##11#11", 1],
        ["1110###", 1],
        ["1#1#011", 1],
        ["0#1#11#", 1],
        ["###11##", 1],
        ["1#10#0#", 1],
        ["1#100##", 1],
        ["#11#1##", 1],
        ["1#1#10#", 1],
        ["101##01", 1],
        ["#011##1", 1],
        ["#1#1#11", 1],
        ["#110#0#", 1],
    ],
    "ABK": [
        ["###0##0", 0],
        ["0#0#0##", 0],
        ["0#####0", 0],
        ["0#01###", 0],
        ["0##10##", 0],
        ["##1#1#1", 1],
        ["1#####1", 1],
        ["1##1###", 1],
        ["##10##1", 1],
        ["###01#1", 1],
    ],
    "DMC": [
        ["01#1#00", 0],
        ["#0#00#0", 0],
        ["#00##00", 0],
        ["0#0###0", 0],
        ["#01#010", 0],
        ["##00000", 0],
        ["01##000", 0],
        ["1#10100", 0],
        ["#0110#1", 0],
        ["0##1100", 0],
        ["#01101#", 0],
        ["#0#0#00", 0],
        ["101110#", 0],
        ["00#0##0", 0],
        ["00##0#1", 0],
        ["#0##100", 0],
        ["00##01#", 0],
        ["#0000##", 0],
        ["#110001", 0],
        ["0#0#0##", 0],
        ["0##00##", 0],
        ["0#01#1#", 0],
        ["1011#01", 0],
        ["1#00101", 0],
        ["1000#0#", 0],
        ["#111000", 0],
        ["#010##0", 0],
        ["0101###", 0],
        ["#011000", 1],
        ["#111#1#", 1],
        ["1#01##1", 1],
        ["#001101", 1],
        ["110#1#0", 1],
        ["110#0#1", 1],
        ["1###111", 1],
        ["0##01#1", 1],
        ["11###1#", 1],
        ["#11#1#1", 1],
        ["1#0#11#", 1],
        ["#11#11#", 1],
        ["1010##1", 1],
        ["00##101", 1],
        ["##1111#", 1],
        ["11#1##1", 1],
        ["0#1#1#1", 1],
        ["##101#1", 1],
        ["1##111#", 1],
        ["01101##", 1],
        ["11100#0", 1],
        ["1101###", 1],
        ["##1#111", 1],
        ["11#11##", 1],
        ["###0111", 1],
        ["1#10#11", 1],
        ["#111##1", 1],
        ["1#01#1#", 1],
    ],
    "COE_1": [
        ["101100#", 0],
        ["#1#1000", 0],
        ["01#10#0", 0],
        ["1#11#00", 0],
        ["#00111#", 0],
        ["#1#0110", 0],
        ["#0000##", 0],
        ["#10010#", 0],
        ["00101##", 0],
        ["0#00##0", 0],
        ["##11100", 0],
        ["00#01#0", 0],
        ["1#0#110", 0],
        ["01010##", 0],
        ["#0#1100", 0],
        ["0#0#0#0", 0],
        ["0##000#", 0],
        ["#1110#0", 0],
        ["#00#001", 0],
        ["00##100", 0],
        ["#00#1#0", 0],
        ["0##0#01", 0],
        ["1#110#0", 0],
        ["##11010", 0],
        ["##001#0", 0],
        ["0##1010", 0],
        ["01##000", 0],
        ["#111#00", 0],
        ["0#0#00#", 0],
        ["10#1001", 0],
        ["00#0#0#", 0],
        ["000###0", 0],
        ["0#00#0#", 0],
        ["#000##0", 0],
        ["0##0110", 0],
        ["#1#0101", 0],
        ["10##101", 1],
        ["###1101", 1],
        ["##11#11", 1],
        ["#1#11#1", 1],
        ["#110100", 1],
        ["101#1#1", 1],
        ["0#11##1", 1],
        ["##1001#", 1],
        ["1#1##11", 1],
        ["#0#1011", 1],
        ["01011##", 1],
        ["1##1011", 1],
        ["1010###", 1],
        ["1#100##", 1],
        ["#11##11", 1],
        ["1##0111", 1],
        ["11#00##", 1],
        ["#111##1", 1],
        ["11###11", 1],
        ["1#10#00", 1],
        ["10010#0", 1],
        ["001100#", 1],
        ["110#01#", 1],
        ["01#111#", 1],
        ["#1#0#11", 1],
        ["##00111", 1],
        ["##1#011", 1],
        ["##1111#", 1],
        ["#1##111", 1],
        ["10#01#1", 1],
        ["11#1##1", 1],
        ["##111#1", 1],
        ["#10110#", 1],
        ["1#0101#", 1],
        ["101#11#", 1],
        ["11##0#1", 1],
    ],
    "COE_2": [
        ["#00##00", 0],
        ["00##000", 0],
        ["010#1##", 0],
        ["1001#0#", 0],
        ["0#0###0", 0],
        ["01#0#11", 0],
        ["#1#001#", 0],
        ["##0000#", 0],
        ["0##0111", 0],
        ["#1000##", 0],
        ["100#00#", 0],
        ["00#000#", 0],
        ["00#011#", 0],
        ["0#0011#", 0],
        ["00101##", 0],
        ["#1##010", 0],
        ["###10#0", 0],
        ["##01#00", 0],
        ["00#1#10", 0],
        ["0010#0#", 0],
        ["00#01#0", 0],
        ["#001##0", 0],
        ["0100###", 0],
        ["##0#0#0", 0],
        ["001#11#", 0],
        ["##0110#", 0],
        ["00##110", 0],
        ["00#0#00", 0],
        ["#11#1#0", 1],
        ["#01001#", 1],
        ["##110#1", 1],
        ["##1110#", 1],
        ["1###111", 1],
        ["#0##011", 1],
        ["1#1#1##", 1],
        ["1##011#", 1],
        ["#1111##", 1],
        ["11##11#", 1],
        ["#111##1", 1],
        ["10###11", 1],
        ["#1#10#1", 1],
        ["#110#0#", 1],
        ["11#01##", 1],
        ["#001#11", 1],
        ["#11#10#", 1],
        ["##11#01", 1],
        ["1010###", 1],
        ["101###1", 1],
        ["1#10#0#", 1],
        ["1##1#11", 1],
        ["0##10#1", 1],
        ["1##01#1", 1],
        ["#000101", 1],
        ["###1011", 1],
        ["1#1##01", 1],
        ["#11##01", 1],
        ["1#11##1", 1],
    ],
    "MM401": [
        ["11#0###", 0],
        ["##1111#", 0],
        ["11####1", 0],
        ["1##01##", 0],
        ["1###1#1", 0],
        ["1#1###1", 0],
        ["1#10###", 0],
        ["1#1#11#", 0],
        ["###1##1", 0],
        ["0##0###", 1],
        ["###1#00", 1],
        ["###10#0", 1],
        ["#00#0#0", 1],
        ["0#0###0", 1],
        ["0####00", 1],
        ["##01##0", 1],
        ["0###0#0", 1],
        ["#0000##", 1],
    ],
}

In [6]:
node = BooleanNode.from_output_list(automata_output_list["GEP_1"])
# plot_schemata(node)

## finding the annihilation and generation rules of a lookup table

This function detects all the annihilation rules (where the state of the automata changes from 1 to 0),
and all the generation rules (0 to 1) of a given wildcard ('#') schemata.  

Filtering these rules directly from wildcard schemata isn't a straightforward process. This function detects it from the output list. It involves removal of annihilation/generation redundancies hidden in the schemata. 

In [38]:
for item in automata_output_list:
    rules = annihilation_generation_rules(automata_output_list[item])
    print(f"{item}: {rules}\n")

GKL: [['###10#0', 0], ['1#10###', 1]]

GP: [['0##1##0', 0], ['0##10##', 0], ['0#01###', 0], ['##10##1', 1], ['1##0##1', 1], ['###01#1', 1]]

GEP_1: [['00#1###', 0], ['0##10##', 0], ['##10##1', 1], ['###0#11', 1]]

GEP_2: [['###1#00', 0], ['##01##0', 0], ['1##01##', 1], ['11#0###', 1]]

Das: [['##01#0#', 0], ['#001###', 0], ['##011#0', 0], ['0#01###', 0], ['###011#', 1], ['#1#01##', 1], ['1#001##', 1], ['###01#1', 1]]

Davis: [['##0100#', 0], ['#1#100#', 0], ['#1#10#0', 0], ['##110#0', 0], ['###1000', 0], ['#0010#1', 0], ['#110#0#', 1], ['1110###', 1], ['0#1011#', 1], ['1#100##', 1], ['1#10#0#', 1], ['#1101##', 1]]

ABK: [['0##1##0', 0], ['0##10##', 0], ['0#01###', 0], ['##10##1', 1], ['1##0##1', 1], ['###01#1', 1]]

DMC: [['#111000', 0], ['#0#1100', 0], ['00#101#', 0], ['0101###', 0], ['00#10#1', 0], ['#0110#1', 0], ['#01101#', 0], ['01#1#00', 0], ['101110#', 0], ['0#010##', 0], ['0#01#1#', 0], ['1011#01', 0], ['0##1100', 0], ['0#01##0', 0], ['#001#00', 0], ['##101#1', 1], ['1#10#11', 

## incomplete automata

used jordan's algorithm and verified with the annihilation generation rules shown in [1].


[1] M. Marques, R. Manurung, and H. Pain, “Conceptual Representations: What do they have to say about the Density Classiﬁcation Task by Cellular Automata?”.


In [8]:
incomplete_automata = {  # keeping annihilation and generation rules only
    "GKL": [["###10#0", 0], ["1#10###", 1]],
    "GP": [
        ["0##1##0", 0],
        ["0##10##", 0],
        ["0#01###", 0],
        ["##10##1", 1],
        ["1##0##1", 1],
        ["###01#1", 1],
    ],
    "GEP_1": [['00#1###', 0], ['0##10##', 0], ['##10##1', 1], ['###0#11', 1]],
    "GEP_2": [['###1#00', 0], ['##01##0', 0], ['1##01##', 1], ['11#0###', 1]],
    "Das": [['##01#0#', 0], ['#001###', 0], ['##011#0', 0], ['0#01###', 0], ['###011#', 1], ['#1#01##', 1], ['1#001##', 1], ['###01#1', 1]],
    "Davis": [['##0100#', 0], ['#1#100#', 0], ['#1#10#0', 0], ['##110#0', 0], ['###1000', 0], ['#0010#1', 0], ['#110#0#', 1], ['1110###', 1], ['0#1011#', 1], ['1#100##', 1], ['1#10#0#', 1], ['#1101##', 1]],
    "ABK": [['0##1##0', 0], ['0##10##', 0], ['0#01###', 0], ['##10##1', 1], ['1##0##1', 1], ['###01#1', 1]],
    "DMC": [['#111000', 0], ['#0#1100', 0], ['00#101#', 0], ['0101###', 0], ['00#10#1', 0], ['#0110#1', 0], ['#01101#', 0], ['01#1#00', 0], ['101110#', 0], ['0#010##', 0], ['0#01#1#', 0], ['1011#01', 0], ['0##1100', 0], ['0#01##0', 0], ['#001#00', 0], ['##101#1', 1], ['1#10#11', 1], ['0##01#1', 1], ['11100#0', 1], ['01101##', 1], ['1#0011#', 1], ['###0111', 1], ['11#0#1#', 1], ['11000#1', 1], ['#11011#', 1], ['11001#0', 1], ['1010##1', 1]],
    "COE_1": [['#00111#', 0], ['##11010', 0], ['01010##', 0], ['#001001', 0], ['1#110#0', 0], ['#1#1000', 0], ['101100#', 0], ['#0011#0', 0], ['1#11#00', 0], ['##11100', 0], ['01#10#0', 0], ['#1110#0', 0], ['#0#1100', 0], ['0#010#0', 0], ['0##1010', 0], ['0001##0', 0], ['10#1001', 0], ['#111#00', 0], ['0#0100#', 0], ['1#01110', 0], ['1#10#11', 1], ['1##0111', 1], ['1010###', 1], ['##00111', 1], ['#110100', 1], ['1#100##', 1], ['11#00##', 1], ['#1#0#11', 1], ['10#01#1', 1], ['1#10#00', 1], ['##1001#', 1]],
    "COE_2": [['00#1#10', 0], ['01011##', 0], ['###10#0', 0], ['##01#00', 0], ['##0110#', 0], ['001111#', 0], ['#001##0', 0], ['1001#0#', 0], ['0#01##0', 0], ['#110#0#', 1], ['#01001#', 1], ['#0#0011', 1], ['1010###', 1], ['1#101##', 1], ['1##011#', 1], ['11#01##', 1], ['#000101', 1], ['1##01#1', 1], ['10#0#11', 1], ['1#10#0#', 1], ['#1101#0', 1]],
    "MM401": [['###1##1', 0], ['##1111#', 0], ['#0000##', 1], ['0##0###', 1]],
}

In [10]:
# investigating the pattern of shuffling

# from itertools import permutations
# import random

# # Without shuffling
# elements = [1, 2, 3, 4]
# # print("Without shuffling:")
# # for perm in permutations(elements):
# #     print(perm)

# # With shuffling
# print("\nWith shuffling:")
# for perm in permutations(elements):
#     perm = list(perm)
#     random.shuffle(perm)
#     print(perm)


In [42]:
node = BooleanNode.from_partial_lut(incomplete_automata["GKL"])
nodes = node.generate_with_required_bias(bias=0.5, verbose=True)
print(concatenate(node.outputs))

6.44e+27 possible permutation(s) with a bias of 0.5. This is the closest bias less than or equal to the required bias of 0.5.
????????0?0?????????????0?0?????????????0?0?????????????0?0?????????????0?0?????111111110?0?????????????0?0?????111111110?0?????


## sampling automata rule from a large space of permutations

In [43]:
check_spread_randomness(nodes, iterations=10000)

[-0.012, -0.0028, 0.0118, -0.0002, -0.0032, -0.0086, 0.0228, -0.0056, -1.0, 0.0052, -1.0, -0.0114, -0.0038, -0.0058, -0.0006, 0.008, 0.0094, 0.0018, 0.0022, -0.017, -0.0032, 0.0124, 0.0006, -0.0094, -1.0, -0.016, -1.0, 0.0168, 0.0136, 0.0098, 0.028, 0.0164, -0.0022, 0.0022, -0.0026, -0.0068, -0.0092, 0.0212, -0.0038, -0.0168, -1.0, -0.0008, -1.0, 0.032, -0.0116, -0.0024, -0.0044, 0.0096, 0.0128, -0.002, 0.005, -0.0038, 0.0132, -0.003, 0.0008, -0.003, -1.0, -0.0114, -1.0, -0.0086, -0.0134, -0.015, -0.0084, -0.0096, -0.0008, 0.0104, 0.008, -0.0018, 0.0012, -0.0126, 0.0042, -0.0032, -1.0, 0.0022, -1.0, 0.0018, 0.0002, -0.0068, 0.0062, 0.0056, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -0.001, -1.0, 0.0012, 0.014, -0.022, -0.0012, 0.0146, 0.012, -0.0178, -0.0132, -0.009, 0.0078, 0.013, -0.0108, 0.0002, -1.0, 0.0058, -1.0, -0.0142, -0.0004, -0.0032, -0.0166, -0.008, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 0.0026, -1.0, 0.0124, -0.0036, 0.0174, -0.0028, -0.009]
Average absolute chan

In [44]:
check_for_duplicates(nodes, iterations=10000)

Total rules generated:10000
Unique rules generated:10000
